Grad-CAM

In [176]:
import os
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, models
import torchvision.utils as vutils
import torchvision.transforms.functional as TF
from PIL import Image

In [177]:
# === Define device ===
device = "cuda" if torch.cuda.is_available() else "cpu"

next cell: recreate architecture to match the one used for training before loading the weights

In [178]:
# === RESNET18 architecture recreation ===
def load_resnet18_model(model_path, num_classes=2, device=torch.device("cpu")):
    # Load base ResNet-18 model
    model = models.resnet18(weights=None)
    # Modify first convolutional layer for grayscale input
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    # Modify the final fully connected layer for binary classification
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2) 
    # Move the model to the correct device (CPU or GPU)
    model.to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    # Set the model to evaluation mode (important for inference)
    model.eval()
    return model

In [179]:
# === Define Grad-CAM ===  
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = None
        self.gradients = None
        self.activations = None
        self.hook_layers(target_layer)

    def hook_layers(self, target_layer): # FP and BP
        def backward_hook(module, grad_input, grad_output):
            #print("[Backward hook called]")
            #print("→ grad_output[0] shape:", grad_output[0].shape)
            self.gradients = grad_output[0] # activations' gradients during backward [1, 512, 3, 3]
        def forward_hook(module, input, output):
            #print("[Forward hook called]")
            #print("→ output shape:", output.shape)
            self.activations = output # layer4 activation during forward [1, 512, 3, 3]


        # Find the layer by name
        for name, module in self.model.named_modules():
            if name == target_layer:
                self.target_layer = module
                module.register_forward_hook(forward_hook)
                module.register_full_backward_hook(backward_hook)
                break

        if self.target_layer is None:
            raise ValueError(f"Layer {target_layer} not found in model.")


    def generate(self, input_tensor, class_indices=None):
        #print(f"[Grad-CAM] Input tensor shape before forward: {input_tensor.shape}")
        output = self.model(input_tensor) # forward pass
        if class_indices is None: # if target class is not specified, it uses class prediction (argmax)
            class_indices = torch.argmax(output, dim=1)

        self.model.zero_grad() # Clear any previous gradients
        one_hot = torch.zeros_like(output, device=device)
        for i, class_idx in enumerate(class_indices):
            one_hot[i, class_idx] = 1

        output.backward(gradient=one_hot) #
        self.model.zero_grad() # Ensure no gradients persist after Grad-CAM computation

         # Compute the Grad-CAM heatmap
        weights = torch.nn.functional.adaptive_avg_pool2d(self.gradients, 1) # avgpool - reduces spatial resolution for output_size=1 -> [1, 512, 1, 1]
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True) # cam operation - [1,1,3,3]
        cam = torch.relu(cam)

        # Normalize Grad-CAM heatmap
        cam = cam / (cam.max(dim=2, keepdim=True)[0].max(dim=3, keepdim=True)[0] + 1e-6)
        return cam.cpu().detach().numpy(), class_indices.cpu().numpy()


In [180]:
# === Define Image Transformations === (Ensure grayscale input and correct format) ************
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((80,80)),  # Match model's input size
    transforms.ToTensor(),
])

In [181]:
# === Extract Ground Truth from folder path ===
def get_ground_truth(img_name, folder):
    # Combine the folder path and image name for a comprehensive check
    full_path = os.path.join(folder, img_name)
    
    # Check for the ground truth in a case-insensitive way
    if "cesarean" in full_path.lower():
        return "Cesarean Birth"
    elif "vaginal" in full_path.lower():
        return "Vaginal Birth"
    else:
        print(f"Warning: Could not determine ground truth for {full_path}. Assuming 'Cesarean Birth'.")
        ground_truth = "Cesarean Birth"

    # Print the ground truth for each image
    #print(f"[Ground Truth] Image: {img_name} | Path: {full_path} | Picked: {ground_truth}")
    return ground_truth

In [182]:
# === Prediction Frame ===
def add_colored_frame(image, color, thickness=20):
    height, width = image.shape[:2]
    cv2.rectangle(image, (0, 0), (width - 1, height - 1), color, thickness)
    return image

In [183]:
# === Heatmap Overlay ===
def overlay_heatmap(original_image, heatmap): #heatmap w\ size [1,3,3]
    heatmap = cv2.resize(heatmap[0], (original_image.shape[1], original_image.shape[0])) #heatmap size from [3,3] to [80,80]
    heatmap = np.uint8(255 * heatmap) #adapt to image format needed for color mapping
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) #apply color map

    if len(original_image.shape) == 2 or original_image.shape[2] == 1:
        original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)

    #original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR) # Convert grayscale to 3-channel image
    overlay = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0) #blend - 60% of grayscale image , 40% for colored heatmap
    return overlay

In [184]:
# === Apply grad-cam in each image from input folder == Apply Frame == Build Grid == Calculate Mean Correct Heatmap ===
def apply_gradcam_and_save(model, input_folder, output_folder, grid_image_size=(80, 80)):
    os.makedirs(output_folder, exist_ok=True)
    grad_cam = GradCAM(model, 'layer4')


    original_images = []
    correct_heatmaps = []
    incorrect_heatmaps = []
    misclassified_tensors = []
    correctly_classified_tensors = []

    class_names = ["Cesarean Birth", "Vaginal Birth"]

    for img_name in os.listdir(input_folder):
        if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(input_folder, img_name)

        # Load image
        image = Image.open(img_path)
        

        # Transform image to tensor: [1, 80, 80] → unsqueeze → [1, 1, 80, 80]
        input_tensor = transform(image).unsqueeze(0).to(device)
        # Save the resized image as original (224x224) to ensure overlay consistency
        resized_image = transforms.Resize((80,80))(image)
        original_np = np.array(resized_image)
        original_images.append(original_np)


        # Grad-CAM
        cam_heatmaps, predicted_classes = grad_cam.generate(input_tensor)
        prediction = predicted_classes[0]
        predicted_label = class_names[prediction]


        # === Ground truth from folder name ===
        ground_truth_label = get_ground_truth(img_name, input_folder)
        ground_truth_class = class_names.index(ground_truth_label)

        # === Choose frame color ===
        frame_color = (0, 255, 0)  # green
        if prediction != ground_truth_class:
            frame_color = (0, 0, 255)  # red

        # === Overlay and save ===
        overlay = overlay_heatmap(original_np, cam_heatmaps[0]) # cam shape = [1,3,3]
        overlay_with_frame = add_colored_frame(overlay, frame_color, thickness=7)

        save_path = os.path.join(output_folder, f"heatmap_{predicted_label}_{img_name}")
        cv2.imwrite(save_path, overlay_with_frame)
        print(f"Saved: {save_path} ; Predicted: {predicted_label}")

        # === Add to grid tensor ===
        overlay_rgb = cv2.cvtColor(overlay_with_frame, cv2.COLOR_BGR2RGB) #converts to rgb
        overlay_pil = Image.fromarray(overlay_rgb).resize(grid_image_size) # downsizes the image to fit the grid
        tensor_for_grid = TF.to_tensor(overlay_pil) #converts into a pytorch tensor [3,H,W] and normalizes pixel values

        if prediction == ground_truth_class:
            correctly_classified_tensors.append(tensor_for_grid)
            correct_heatmaps.append(cam_heatmaps[0][0])  # 2D spatial heatmap , cam shape = [3,3]
        else:
            misclassified_tensors.append(tensor_for_grid)
            incorrect_heatmaps.append(cam_heatmaps[0][0])

    # === CREATE GRID ===
    final_sorted_grid = misclassified_tensors + correctly_classified_tensors
    if final_sorted_grid:
        grid = vutils.make_grid(final_sorted_grid, nrow=20, padding=5, normalize=True)
        grid_path = os.path.join(output_folder, "grid_overlay_ordered.jpg")
        vutils.save_image(grid, grid_path)
        print(f"Grid saved at: {grid_path}")

    # === CALCULATE CORRECT MEAN HEATMAP ===
    if correct_heatmaps: #list of NumPy arrays (raw grad-cam maps before resizing or coloring), each w\ shape [3, 3]
        mean_heatmap = np.mean(np.stack(correct_heatmaps), axis=0) # turns list of N heatmaps into 3D array w\ shape [N, 3, 3]
        mean_heatmap -= mean_heatmap.min() # subtracts the minimum value from every pixel so the lowest value is 0
        mean_heatmap /= (mean_heatmap.max() + 1e-8) # divides every pixel by the new maximum value (normalize) ; adds 1e-8 to ensure it wont be divided by 0
        heatmap_uint8 = np.uint8(255 * mean_heatmap) # Converts [0.0 – 1.0] → [0 – 255] format needed for OpenCV
        heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET) # assigns color to the values

        # Resize to match original image size
        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_NEAREST) #INTER_CUBIC for smoother transitions ; INTER_NEAREST for blocky look
        heatmap_resized_smooth = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_CUBIC)


        mean_path_blocky = os.path.join(output_folder, "correct_mean_heatmap_blocky.png")
        cv2.imwrite(mean_path_blocky, heatmap_resized_blocky)
        print(f"Mean heatmap (correct only) saved at: {mean_path_blocky}")

        mean_path_smooth = os.path.join(output_folder, "correct_mean_smooth.png")
        cv2.imwrite(mean_path_smooth, heatmap_resized_smooth)
        print(f"Mean heatmap (correct only) saved at: {mean_path_smooth}")

    # === CALCULATE INCORRECT MEAN HEATMAP ===
    if incorrect_heatmaps:
        mean_heatmap_incorrect = np.mean(np.stack(incorrect_heatmaps), axis=0)
        mean_heatmap_incorrect -= mean_heatmap_incorrect.min()
        mean_heatmap_incorrect /= (mean_heatmap_incorrect.max() + 1e-8)
        heatmap_uint8_incorrect = np.uint8(255 * mean_heatmap_incorrect)
        heatmap_colored_incorrect = cv2.applyColorMap(heatmap_uint8_incorrect, cv2.COLORMAP_JET)

        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_NEAREST)
        heatmap_resized_smooth_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_CUBIC)

        mean_path_blocky_incorrect = os.path.join(output_folder, "incorrect_mean_heatmap_blocky.png")
        cv2.imwrite(mean_path_blocky_incorrect, heatmap_resized_blocky_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_blocky_incorrect}")

        mean_path_smooth_incorrect = os.path.join(output_folder, "incorrect_mean_smooth.png")
        cv2.imwrite(mean_path_smooth_incorrect, heatmap_resized_smooth_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_smooth_incorrect}")


    return correct_heatmaps, original_images

In [197]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Abdomen_/test/Cesarean Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_test_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_abdomen_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU1003212_abdomen1_282.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU11011732_abdomen1_13.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU11018785_abdomen1_16.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU12009721_abdomen1_286.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/

In [186]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Abdomen_/train/Cesarean Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_train_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_abdomen_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10002107_abdomen1_276.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10008343_abdomen1_278.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10011068_abdomen1_467.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10022513_abdomen1_1.png ; Predicted: Cesarean Birth
Saved: C:/Users/an

In [187]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Abdomen_/test/Vaginal Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_test_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_abdomen_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10004004_abdomen1_0.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10004597_abdomen1_203.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10010378_abdomen1_636.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10012122_abdomen1_0.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Docum

In [188]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Abdomen_/train/Vaginal Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_train_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_abdomen_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10000998_abdomen1_201.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10012069_abdomen1_207.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10016707_abdomen1_4.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_ABDOMEN_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10017095_abdomen1_637.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive

In [189]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Femur_/test/Cesarean Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_test_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_femur_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU1003212_femur1_349.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_test_cv2_layer4\heatmap_Cesarean Birth_PU11011732_femur1_11.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU11018785_femur1_15.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_test_cv2_layer4\heatmap_Cesarean Birth_PU12009721_femur1_353.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/U

In [190]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Femur_/train/Cesarean Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_train_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_femur_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10002107_femur1_346.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10008343_femur1_347.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10011068_femur1_493.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10022513_femur1_160.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/D

In [191]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Femur_/train/Vaginal Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_train_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_femur_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10000998_femur1_878.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10012069_femur1_884.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10016707_femur1_3.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10017095_femur1_617.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Univ

In [192]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Femur_/test/Vaginal Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_test_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_femur_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10004004_femur1_0.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10004597_femur1_879.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10010378_femur1_616.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_FEMUR_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10012122_femur1_159.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universi

In [193]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Head_/test/Cesarean Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_test_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_head_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU1003212_head1_201.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU11011732_head1_8.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU11018785_head1_9.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_test_cv2_layer4\heatmap_Vaginal Birth_PU12009721_head1_202.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TE

In [194]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Head_/train/Cesarean Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_train_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_head_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10002107_head1_199.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10008343_head1_200.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10011068_head1_313.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_cesarean_train_cv2_layer4\heatmap_Cesarean Birth_PU10022513_head1_144.png ; Predicted: Cesarean Birth
Saved: C:/Users/anale/OneDrive/Documento

In [195]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Head_/test/Vaginal Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_test_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_head_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10004004_head1_416.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10004597_head1_580.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10010378_head1_328.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_test_cv2_layer4\heatmap_Vaginal Birth_PU10012122_head1_0.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TES

In [196]:
if __name__ == "__main__":
    input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_2/Head_/train/Vaginal Birth'
    output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_train_cv2_layer4'
    model_path = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/ResNet_paths/best-model_cv2_head_resnet18.pth'

    # Run the Grad-CAM processing and save grid

    model = load_resnet18_model(model_path, num_classes=2)
    all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)
    print("Grad-CAM heatmaps saved successfully.")

C:\Users\anale\AppData\Local\Temp\ipykernel_12264\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10000998_head1_579.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10012069_head1_585.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10016707_head1_418.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/80X80_RESNET18/results_80x80_GradCAM_cv2dataset_RESNET18/X_80x80_HEAD_vaginal_train_cv2_layer4\heatmap_Vaginal Birth_PU10017095_head1_329.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universida